<a href="https://colab.research.google.com/github/SivaKumarRajuPaidi/PyTorch/blob/main/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

#Download test data from open Dataset
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [3]:
batch_size = 64

# Create Data Loaders

train_dataloader = DataLoader(dataset = training_data, batch_size = batch_size)
test_dataloader = DataLoader(dataset = test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"Shape of X[ N, C, H, W ] : {X.shape}  {X.dtype}")
  print(f"Shape of y: {y.shape}  {y.dtype}")
  break

Shape of X[ N, C, H, W ] : torch.Size([64, 1, 28, 28])  torch.float32
Shape of y: torch.Size([64])  torch.int64


In [4]:
print(len(train_dataloader.dataset))
print(len(test_dataloader.dataset))
print(len(train_dataloader))
print(len(test_dataloader))

60000
10000
938
157


In [32]:
938*64

60032

In [5]:
# Get CPU, GPU or MPS for training
device = ("cuda"
          if torch.cuda.is_available()
          else "mps"
          if torch.backends.mps.is_available()
          else
          "cpu"
)
print(f"Device being used is {device}")

Device being used is cpu


In [6]:
# Define Model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# Optimizing model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)


        # Backpropogation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss : {loss:>7f} [{current:>5d} / {size:>5d}] ")



In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X , y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch : {t+1}\n ------------------------------")
    train(train_dataloader,model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch : 1
 ------------------------------
loss : 2.290810 [   64 / 60000] 
loss : 2.280527 [ 6464 / 60000] 
loss : 2.264486 [12864 / 60000] 
loss : 2.267437 [19264 / 60000] 
loss : 2.244921 [25664 / 60000] 
loss : 2.210945 [32064 / 60000] 
loss : 2.215770 [38464 / 60000] 
loss : 2.175425 [44864 / 60000] 
loss : 2.179784 [51264 / 60000] 
loss : 2.144480 [57664 / 60000] 
Test Error: 
 Accuracy: 47.7%, Avg loss: 2.139019 

Epoch : 2
 ------------------------------
loss : 2.139904 [   64 / 60000] 
loss : 2.126277 [ 6464 / 60000] 
loss : 2.070533 [12864 / 60000] 
loss : 2.105166 [19264 / 60000] 
loss : 2.029805 [25664 / 60000] 
loss : 1.965989 [32064 / 60000] 
loss : 2.001528 [38464 / 60000] 
loss : 1.904903 [44864 / 60000] 
loss : 1.928112 [51264 / 60000] 
loss : 1.857390 [57664 / 60000] 
Test Error: 
 Accuracy: 54.8%, Avg loss: 1.847095 

Epoch : 3
 ------------------------------
loss : 1.868599 [   64 / 60000] 
loss : 1.833824 [ 6464 / 60000] 
loss : 1.718919 [12864 / 60000] 
loss : 1.78

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
print(output)
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

tensor(2.5475, grad_fn=<NllLossBackward0>)


In [ ]:
output.item()

2.1411476135253906

In [13]:
a = torch.randn(4, 4)
print(a)


tensor([[-0.2271,  1.6934, -0.0632,  1.5879],
        [ 0.6229,  1.0794, -0.4349, -0.3003],
        [-1.2225,  1.4618, -0.6772,  0.7042],
        [-2.8406, -0.3516, -1.1027,  0.7663]])


In [14]:
print(torch.argmax(a))

tensor(1)


In [17]:
a.argmax(1)

tensor([1, 1, 1, 3])

In [23]:
(a.argmax(1) == torch.tensor([1, 1, 1, 3])).type(torch.float).sum().item()

4.0

In [27]:
len(train_dataloader)

938

In [30]:
len(train_dataloader.dataset)

60000